In [ ]:
%matplotlib tk 
import mne
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/media/mangaldeep/HDD2/workspace/MotionControl_MasterThesis')
from main.extraction.physionet_MI import extractPhysionet
from main.extraction.bci3_IVa import extractBCI3
from data import brain_atlas  as bm
plot = False

### Physionet

In [ ]:
# raw = extractPhysionet(runs = [3], person_id = 3)
# # raw.pick_channels(['C3','C4'])
# raw.notch_filter(60)
# raw = raw.filter(7,50,verbose = False).copy()

# event_data = mne.events_from_annotations(raw)
# event_marker, event_ids = event_data
# event_ids = dict({'T1':2, 'T2':3})
# epochs = mne.Epochs(raw, event_marker, event_ids, tmin=-2, tmax=4, preload = True)

In [ ]:
# epoch_file = '../preproc/Physionet_trial_epo.fif'
# epochs = mne.read_epochs(epoch_file)

In [ ]:
# epochs = epochs.apply_proj().copy()
# epochs = epochs.apply_baseline((-2.0,-0.2)).copy()
# epochs.equalize_event_counts()
# tmin, tmax = -0.5, 2
# epochs.crop(tmin= tmin, tmax=tmax)
# picks=['C1', 'C2','C3', 'C4', 'C5', 'C6', 'Cz']#,'T9','T10']
# epochs.pick_channels(picks)
# T1 = epochs['T1']
# T2 = epochs['T2']
# t1 = epochs['T1'].average()
# t2 = epochs['T2'].average()
# # mne.viz.plot_compare_evokeds([T1,-T2],picks= c);
# classes = list(event_ids.keys())

### BCI3 -IVa

In [ ]:
# raw = extractBCI3(runs = 0, person_id = 3)
# raw = raw.filter(7,30,verbose = False).copy()

# event_data = mne.events_from_annotations(raw)
# event_marker, event_ids = event_data
# event_ids = dict(right = 1, foot = 3, test = 2)
# epochs = mne.Epochs(raw, event_marker, event_ids, tmin=-0.5, tmax=1, preload = True)

In [ ]:
epoch_file = '../preproc/BCI3_ssp_ica_[3]_P3_epo.fif'
epochs = mne.read_epochs(epoch_file)
event_ids = epochs.event_id

In [ ]:
epochs = epochs.apply_proj().copy()
epochs = epochs.apply_baseline().copy()
epochs.equalize_event_counts()
tmin, tmax = -0.5, 2
# epochs.crop(tmin= tmin, tmax=tmax)
picks=['C1', 'C2','C3', 'C4', 'C5', 'C6', 'Cz']#,'T9','T10']
epochs.pick_channels(picks)
T1 = epochs['right']
T2 = epochs['foot']
t1 = epochs['right'].average()
t2 = epochs['foot'].average()
classes = list(event_ids.keys())
# mne.viz.plot_compare_evokeds([T1,-T2],picks= c);

### Epochs Analysis

In [ ]:
epochs

In [ ]:
# epochs.plot_image();
# epochs.plot_psd();
# scale = dict(mag=1e-12, grad=4e-11, eeg=100e-6)
# epochs.plot(scalings = scale);

# t1.plot_joint(times = (0.19,2.47));

# t2.plot_joint(times = (0.19,2.47));

In [ ]:
epochs_data = epochs.get_data()
# labels = epochs.events[epochs.events[:,-1]!=3]
labels = epochs.events[:,-1]
labels = labels-2
# labels = epochs.events[:,-1]-2
print(f"Data Size: {epochs_data.shape}")

# train data 
tmin, tmax = -0.5, 1
epochs_train = epochs.copy().crop(tmin= tmin, tmax=tmax)
epochs_train_data = epochs_train.get_data()
# epochs_train_data = np.transpose(epochs_train_data, axes = (1, 2, 0)) # Sklearn data = n_sam * n_freq
print(f"Train Size: {epochs_train_data.shape}")

In [ ]:
if plot == True:
    clas = 'foot'
    # epochs['right'].plot_image(combine = 'mean', title = 'right');
    epochs[clas].average().plot_image(show_names = 'all', titles = clas, picks = picks);
    epochs[clas].plot_topo_image(title = clas);
    epochs[clas].plot_image(title = clas, combine = 'mean');
    epochs[clas].plot_psd();
    epochs[clas].average().plot_joint(picks=picks);

### Power

In [ ]:
from mne.time_frequency import tfr_morlet

In [ ]:
frequencies =  np.arange(5,30,1) #np.logspace(*np.log10([5, 30]), num=25)
chpicks = [1] #baseline = (-0.5, 0.),

In [ ]:
power = tfr_morlet(epochs[classes[0]], freqs = frequencies, n_cycles = frequencies/2, return_itc= False)
if plot == True:
    for pick in chpicks:
        # power.plot(picks=[pick],  vmin=-0.00003, vmax=0.00003, title=f'{pick}')#, mode='logratio')
        # power.plot_joint(timefreqs = [(0.19,10),(2.47,10)],mode = 'logratio', title = classes[0])
        power.plot_joint(mode = 'mean', title = classes[0])

In [ ]:
power = tfr_morlet(epochs[classes[1]], freqs = frequencies, n_cycles = frequencies/2, return_itc= False)
if plot == True:
    for pick in chpicks:
        # power.plot(picks=[pick],  vmin=-0.00003, vmax=0.00003, title=f'{pick}')#, mode='logratio')
        power.plot_joint(timefreqs = [(0.19,10),(2.47,10)],mode = 'logratio', title = classes[1])

In [ ]:
# 1/f removal- Power Normalization - Average TFR
clas = 'right'
baset = [epochs.baseline[1] , -0.2]
power = tfr_morlet(epochs[clas], freqs = frequencies, n_cycles = frequencies/2, return_itc= False)
powerDB = np.zeros(power.data.shape)
t = np.arange(np.abs(baset[1]-baset[0])*epochs.info['sfreq'], dtype = np.int64)
for ch in range(power.data.shape[0]):
    for f in range(power.data.shape[1]):
        baseline = np.mean(power.data[ch,f,t])
        activity = power.data[ch,f,:]
        powerDB[ch,f,:] = 10*np.log10(activity/baseline)

newPower = mne.time_frequency.AverageTFR(epochs.info, powerDB,epochs.times, frequencies,1).copy()
if plot == True:
    newPower.plot_joint(title = clas, mode = 'mean');

In [ ]:
newPower.data.shape

In [ ]:
# 1/f removal- Power Normalization - Epochs TFR
def powerEPdBcalc(clas):
    baset = [epochs.baseline[1] , -0.2]
    power = tfr_morlet(epochs[clas], freqs = frequencies, n_cycles = frequencies/2, return_itc= False, average=False)
    shp = power.data.shape
    powerdB = np.zeros((shp[0],shp[2],shp[3]))
    t = np.arange(np.abs(baset[1]-baset[0])*epochs.info['sfreq'], dtype = np.int64)
    for ep in range(power.data.shape[0]):
        EPpower  = np.mean(power.data[ep,:,:,:],axis=1)
        for f in range(EPpower.shape[0]):
            baseline = np.mean(EPpower[f,t])
            activity = EPpower[f,:]
            powerdB[ep,f,:] = 10*np.log10(activity/baseline)
    return powerdB

# newEPPower = tfr_morlet(epochs[clas], freqs = frequencies, n_cycles = frequencies/2, return_itc= False, average=False)
powerEpdB = {}
powerEpdB[classes[0]] = powerEPdBcalc(classes[0])
powerEpdB[classes[1]] = powerEPdBcalc(classes[1])
# newPower.plot_joint(title = clas, mode = 'mean');

In [ ]:
train_data = np.vstack((powerEpdB['right'],powerEpdB['foot']))
train_data = train_data.reshape(train_data.shape[0],-1)
labels = np.concatenate([-np.ones(powerEpdB[classes[0]].shape[0]),
                     np.ones(powerEpdB[classes[1]].shape[0])])
print(train_data.shape)
print(labels.shape)

In [ ]:
# import matplotlib.pyplot as plt
# powerDB.shape #[f,t]
# plt.figure()
# plt.imshow(powerDB[1,:,:])
# plt.show

### Classification

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
clf = LDA()
cv = ShuffleSplit(10, test_size = 0.2, random_state=1)
cv_split = cv.split(train_data, labels)
scaler = MinMaxScaler((0,1))
train_data = scaler.fit_transform(train_data)

In [ ]:
# pipe = Pipeline([('scatter', scat_obj), ('clf', clf)])
# scores = cross_val_score(pipe, epochs_train_data, labels, cv = cv, verbose=False)

In [ ]:
sfreq = epochs.info['sfreq']
w_length = int(sfreq * 0.05)   # running classifier: window length
w_step = int(sfreq * 0.01)  # running classifier: window step size
w_start = np.arange(0, train_data.shape[1] - w_length, w_step)

scores_windows = []

for train_idx, test_idx in cv_split:
    y_train, y_test = labels[train_idx], labels[test_idx]

    X_train = train_data[train_idx]
    X_test = train_data[test_idx]

    # fit classifier
    clf.fit(X_train, y_train)

    # running classifier: test classifier on sliding window
    score_this_window = []
    # for n in w_start:
    #     X_test = (train_data[test_idx]) #[:, n:(n + w_length)])
    scores_windows.append(clf.score(X_test, y_test))
    # scores_windows.append(score_this_window)
w_times = (w_start + w_length / 2.) / sfreq + epochs.tmin
np.mean(scores_windows)
# plt.scatter(scores_windows)